# Used Car Price Prediction

The goal of this notebook is to predict the price of a used vehicle given 12 distinct features such as brand, model, mileage etc.

Using the dataset, i build visualizations to understand the underlying factors affecting the price of a used car. Finally we shall build a model to predict the final price of a used car.

# Imports

Here i install and import all the libraries neccessary to run this notebook

In [2]:
%pip install catboost
%pip install optuna
%pip install optuna_distributed
%pip install openfe
%pip install seaborn
%pip install -q -U autogluon.tabular
%pip install -Uqq fastbook
import fastbook
%fastbook.setup_book()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%fastbook.setup_book()` not found.


In [3]:
from fastbook import *

from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB

#transformers and pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv

import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from optuna.samplers import TPESampler
import warnings

matplotlib.rc('image', cmap='Greys')

#from fastkaggle import setup_comp

from openfe import OpenFE, transform

from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [9]:
!ls

LICENSE      kaggle_x.ipynb	    used-car-price-prediction-dataset.zip
README.md    kaggle_x.ipynb.amltmp
kaggle.json  kagglex-cohort4.zip


In [6]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

#!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/home/azureuser/.kaggle’: File exists


# Download Datasets

In [7]:
!kaggle competitions download -c kagglex-cohort4

 46%|█████████████████▌                    | 1.00M/2.17M [00:00<00:00, 8.93MB/s]
100%|██████████████████████████████████████| 2.17M/2.17M [00:00<00:00, 8.38MB/s]


### Original dataset

In [8]:
!kaggle datasets download -d taeefnajib/used-car-price-prediction-dataset

Dataset URL: https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
  0%|                                                | 0.00/109k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 109k/109k [00:00<00:00, 1.30MB/s]


In [15]:
!ls dataset/original

In [12]:
!mkdir dataset
!chmod +w dataset
!mkdir dataset/original
!chmod +w dataset/original

In [16]:
!unzip kagglex-cohort4.zip -d dataset
!unzip used-car-price-prediction-dataset.zip -d dataset/original/

Archive:  kagglex-cohort4.zip
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/train.csv       
Archive:  used-car-price-prediction-dataset.zip
  inflating: dataset/original/used_cars.csv  


In [18]:
!ls dataset/original

used_cars.csv


In [19]:
path = Path('dataset/')
path

Path('dataset')

# Define DataFrames

In [20]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')
original_df = pd.read_csv(path/'original/used_cars.csv')

In [21]:
train_df.head(2)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoline Fuel,6-Speed M/T,Black,Black,None reported,Yes,8250


In [22]:
original_df.head(2)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"


Original df price has a $ as the train_df doesnt

In [23]:
train_df.shape,test_df.shape,original_df.shape

((54273, 12), (36183, 11), (4009, 12))

In [24]:
train_df.columns,test_df.columns

(Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
        'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
        'price'],
       dtype='object'),
 Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
        'transmission', 'ext_col', 'int_col', 'accident', 'clean_title'],
       dtype='object'))

# Baseline

In [25]:
cont_names,cat_names = cont_cat_split(train_df, dep_var='price')
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='price',
                   splits=splits)

X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

dls = to.dataloaders(bs=64)
test_dl = dls.test_dl(test_df)

# Optuna

In [ ]:
%%time
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1200),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15)
    }
    
    model = LGBMClassifier(**params, objective='multiclass', random_state=0, device='cpu', verbosity=-1)
    
    # Cross-validation with 5 folds using KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_results = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')
    
    # We maximize accuracy, so we return the mean accuracy of the cross-validation
    return np.mean(cv_results)

study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

In [12]:
%%time
import numpy as np
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import optuna

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1200),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15)
    }
    
    model = LGBMClassifier(**params, objective='regression', random_state=0, device='cpu', verbosity=-1)
    
    # Cross-validation with 5 folds using KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_results = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')  # Use neg_mean_squared_error for RMSE
    
    # We minimize RMSE, so we return the negative mean squared error (to convert it to a maximization problem)
    return -np.mean(cv_results)

study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)


/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-06-23 19:53:25,243] A new study created in memory with name: no-name-50970b2a-d4a6-4a41-99c8-8306af1a676d
[I 2024-06-23 21:13:53,573] Trial 0 finished with value: 222051582.47651833 and parameters: {'num_leaves': 320, 'learning_rate': 0.269388301928541, 'n_estimators': 843, 'subsample_for_bin': 172567, 'min_child_samples': 223, 'reg_alpha': 0.0028770084050677926, 'reg_lambda': 2.3761404778025532e-05, 'colsample_bytree': 0.9350638004692479, 'subsample': 0.9727470703757719, 'max_depth': 6}. Best is trial 0 with value: 222051582.47651833.
[I 2024-06-23 22:32:41,836] Trial 1 finished with value: 254752551.68751606 and parameters: {'num_leaves': 417, 'learning_rate': 0.11423254155608371, 'n_estimators': 811, 'subsample_for_bin': 279167, 'min_child_samples': 

In [ ]:
print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)


# Models

In [27]:
%%time

rf = RandomForestRegressor(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))

rf_preds_x = tensor(rf_model.predict(X_test))

mse = mean_squared_error(y_test, rf_preds_x)
rmse = np.sqrt(mse)
print(rmse)


58749.51853846713
CPU times: user 17.4 s, sys: 42.4 ms, total: 17.4 s
Wall time: 17.4 s


In [28]:
mse = mean_squared_error(y_test, rf_preds_x)
rmse = np.sqrt(mse)
print(rmse)


74318.74025617407


In [28]:
%%time
cat_model = CatBoostRegressor(iterations=2000, depth=8, learning_rate=  0.08, random_strength=10)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))

mse = mean_squared_error(y_test, cat_preds_x)
rmse = np.sqrt(mse)
print(rmse)

56560.31252503457
CPU times: user 38.3 s, sys: 3.62 s, total: 42 s
Wall time: 13.2 s


In [29]:
xgb_model = xgb.XGBRegressor(n_estimators = 197, max_depth=4, learning_rate=0.1818695751227044, subsample= 0.39774994666482544)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

mse = mean_squared_error(y_test, xgb_preds_x)
rmse = np.sqrt(mse)
print(rmse)

58988.18613881335


In [30]:
lgb_model = lgb.LGBMRegressor(num_leaves=251, learning_rate=0.02956613668999794, n_estimators=483, max_depth=82, boosting_type='gbdt',min_child_samples=90, random_state=27)
lgb_model = lgb_model.fit(X_train, y_train)

#test set preds
lgb_preds = tensor(lgb_model.predict(test_dl.xs))

#validation set preds
lgb_preds_x = tensor(lgb_model.predict(X_test))

mse = mean_squared_error(y_test, lgb_preds_x)
rmse = np.sqrt(mse)
print(rmse)

57481.14285691434


In [32]:
#lgb_preds_x.shape,rf_preds_x.shape ,cat_preds_x.shape ,xgb_preds_x.shape,general_preds.shape

In [ ]:
general_preds = (lgb_preds_x + rf_preds_x + cat_preds_x + xgb_preds_x)/4
general_preds

In [ ]:
general_preds_sub = (lgb_preds + rf_preds + cat_preds + xgb_preds)/4
general_preds_sub

In [ ]:
general_preds_sub.shape,cat_preds.shape,lgb_preds.shape

In [ ]:
mse = mean_absolute_error(y_test, general_preds)
rmse = np.sqrt(mse)
print(rmse)

In [33]:
submit = pd.read_csv(path/'sample_submission.csv')
submit.shape,cat_preds.shape

((36183, 2), torch.Size([36183]))

In [33]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['price'] = rf_preds
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

,id,price
0,54273,21217.232
1,54274,18184.297
2,54275,28220.502
3,54276,74072.016
4,54277,40165.652
...,...,...
36178,90451,93780.664
36179,90452,9515.303
36180,90453,10171.375
36181,90454,58793.305


In [34]:
!kaggle competitions submit -c kagglex-cohort4 -f submission.csv -m "rf_preds baseline"

100%|████████████████████████████████████████| 552k/552k [00:00<00:00, 1.65MB/s]
400 - Bad Request - Submission not allowed:  Submissions have been disabled for this competition.


### This initial submission gave me a score of 76404.555

In [ ]:
dls = to.dataloaders(bs=64)

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)
#learn.lr_find(suggest_funcs=(slide,valley))

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
%%time
learn.fit_one_cycle(10,0.012)

In [ ]:
%%time
dl = learn.dls.test_dl(test_df)
nn_preds = learn.get_preds(dl=dl)
nn_preds_x = learn.get_preds()[0]
a_preds, _ = learn.get_preds(dl=dl)
nn_preds_y = a_preds.squeeze(1)
nn_preds_m = nn_preds_x.squeeze(1)
nn_preds_z = nn_preds[0]

#target_preds = nn_preds[0]

#test_df['Rings'] = target_preds
#test_df.to_csv('submission.csv', columns=['Rings'], index=True)

#submission = pd.read_csv('submission.csv')
#submission.head()

In [ ]:
nn_preds_x.shape,nn_preds_y.shape,nn_preds_m.shape

In [ ]:
mse = mean_absolute_error(y_test, nn_preds_x)
rmse_nn = np.sqrt(mse)
print(rmse_nn)

In [ ]:
mse = mean_absolute_error(y_test, nn_preds_m)
rmse_nn = np.sqrt(mse)
print(rmse_nn)

# Adding Original Dataset

In [10]:
train_final = pd.concat([train_df,original_df], axis=0)
train_final.shape

(58282, 12)

In [12]:
cont_names,cat_names = cont_cat_split(train_final, dep_var='price')
splits = RandomSplitter(valid_pct=0.2)(range_of(train_final))
to = TabularPandas(train_final, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='price',
                   splits=splits)

X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

dls = to.dataloaders(bs=64)
test_dl = dls.test_dl(test_df)

In [13]:
%%time
rf = RandomForestRegressor(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))

rf_preds_x = tensor(rf_model.predict(X_test))

mse = mean_absolute_error(y_test, rf_preds_x)
rmse = np.sqrt(mse)

mse = mean_absolute_error(y_test, rf_preds_x)
rmse = np.sqrt(mse)
print(rmse)


CPU times: user 18.7 s, sys: 43.5 ms, total: 18.7 s
Wall time: 18.7 s


In [14]:
mse = mean_absolute_error(y_test, rf_preds_x)
rmse = np.sqrt(mse)
print(rmse)


12.786885


In [15]:
%%time
cat_model = CatBoostRegressor(iterations=2000, depth=8, learning_rate=  0.08, random_strength=10)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))

cat_mse = mean_absolute_error(y_test, cat_preds_x)
cat_rmse = np.sqrt(cat_mse)
print(cat_rmse)

12.714128
CPU times: user 42.3 s, sys: 2.76 s, total: 45.1 s
Wall time: 13.4 s


In [18]:
xgb_model = xgb.XGBRegressor(n_estimators = 197, max_depth=4, learning_rate=0.1818695751227044, subsample= 0.39774994666482544)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

xgb_mse = mean_absolute_error(y_test, xgb_preds_x)
xgb_rmse = np.sqrt(xgb_mse)
print(xgb_rmse)

12.814821


In [16]:
lgb_model = lgb.LGBMRegressor(num_leaves=251, learning_rate=0.02956613668999794, n_estimators=483, max_depth=82, boosting_type='gbdt',min_child_samples=90, random_state=27)
lgb_model = lgb_model.fit(X_train, y_train)

#test set preds
lgb_preds = tensor(lgb_model.predict(test_dl.xs))

#validation set preds
lgb_preds_x = tensor(lgb_model.predict(X_test))

lgb_mse = mean_absolute_error(y_test, lgb_preds_x)
lgb_rmse = np.sqrt(lgb_mse)
print(lgb_rmse)

12.719144


In [20]:
general_preds = (lgb_preds_x + rf_preds_x + cat_preds_x + xgb_preds_x)/4
general_preds

tensor([ 730.1823,  609.2162,  215.0769,  ...,  568.4324, 1035.6805,
         551.1803])

In [21]:
mse = mean_absolute_error(y_test, general_preds)
rmse = np.sqrt(mse)
print(rmse)

12.638833


In [23]:
general_preds_sub = (lgb_preds + rf_preds + cat_preds + xgb_preds)/4
general_preds_sub

tensor([381.6687, 367.4169, 516.5643,  ..., 141.9362, 956.5161, 195.2933])

In [25]:
!rm submission.csv

In [28]:
!ls

AutogluonModels  kaggle.json		kagglex-cohort4.zip
catboost_info	 kaggle_x.ipynb		submission.csv
dataset		 kaggle_x.ipynb.amltmp	used-car-price-prediction-dataset.zip


In [27]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['price'] = general_preds_sub
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

,id,price
0,54273,381.66870
1,54274,367.41690
2,54275,516.56430
3,54276,944.34140
4,54277,624.46875
...,...,...
36178,90451,1008.12840
36179,90452,155.77377
36180,90453,141.93616
36181,90454,956.51605


In [29]:
!kaggle competitions submit -c kagglex-cohort4 -f submission.csv -m "general_preds baseline + original dataset"

100%|████████████████████████████████████████| 551k/551k [00:00<00:00, 1.72MB/s]
Successfully submitted to KaggleX Skill Assessment Challenge

## AutoML - AutoGluon

In [ ]:
%pip install AutoGluon

In [30]:
%%time
predictor = TabularPredictor(label='price').fit(train_data=train_df)

No path specified. Models will be saved in: "AutogluonModels/ag-20240607_035321"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240607_035321"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.8.5
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #47~20.04.1-Ubuntu SMP Fri Jun 2 21:38:08 UTC 2023
CPU 

CPU times: user 10min 5s, sys: 4.38 s, total: 10min 10s
Wall time: 3min 57s


In [31]:
%%time
predictions = predictor.predict(test_df)

CPU times: user 2.52 s, sys: 39.9 ms, total: 2.56 s
Wall time: 2.14 s


In [32]:
predictions

id
54273    23219.078125
54274    23511.646484
54275    40694.859375
54276    56333.109375
54277    36551.843750
             ...     
90451    70927.515625
90452    52173.867188
90453    13193.083984
90454    55341.703125
90455    24132.984375
Name: price, Length: 36183, dtype: float32

In [33]:
%%time
predictor.evaluate(train_df)

CPU times: user 3.93 s, sys: 52.2 ms, total: 3.98 s
Wall time: 2.95 s


{'root_mean_squared_error': -65665.22506090587,
 'mean_squared_error': -4311921782.299419,
 'mean_absolute_error': -16564.72589797219,
 'r2': 0.18697863036499762,
 'pearsonr': 0.449699835387118,
 'median_absolute_error': -8163.80859375}

In [42]:
!rm submission.csv

In [43]:
!ls

AutogluonModels  kaggle.json		kagglex-cohort4.zip
catboost_info	 kaggle_x.ipynb		used-car-price-prediction-dataset.zip
dataset		 kaggle_x.ipynb.amltmp


Investigate if there is a difference in score based on the submission steps used.

In [37]:
#currently preffered submission style
submit = pd.read_csv(path/'sample_submission.csv')
submit['price'] = predictions
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

,id,price
0,54273,NaN
1,54274,NaN
2,54275,NaN
3,54276,NaN
4,54277,NaN
...,...,...
36178,90451,NaN
36179,90452,NaN
36180,90453,NaN
36181,90454,NaN


In [44]:
#seems to work better for submissions
test_df['price'] = predictions
test_df.to_csv('submission.csv', columns=['price'], index=True)

submission = pd.read_csv('submission.csv')
submission.head()

,id,price
0,54273,23219.078
1,54274,23511.646
2,54275,40694.860
3,54276,56333.110
4,54277,36551.844


In [46]:
!kaggle competitions submit -c kagglex-cohort4 -f submission.csv -m "AutoGluon Baseline"

100%|████████████████████████████████████████| 553k/553k [00:00<00:00, 1.94MB/s]
Successfully submitted to KaggleX Skill Assessment Challenge

In [45]:
!ls

AutogluonModels  kaggle.json		kagglex-cohort4.zip
catboost_info	 kaggle_x.ipynb		submission.csv
dataset		 kaggle_x.ipynb.amltmp	used-car-price-prediction-dataset.zip


In [ ]:
from openfe import OpenFE, transform

ofe = OpenFE()

In [ ]:
y_names='Target'